# LangGraph Graph Compilation & Testing

Compile, invoke, and test your LangGraph workflow with different scenarios.

## Learning Objectives

By the end of this notebook, you will:

1. **Understand compilation** — What `compile()` does and why it's required
2. **Invoke the graph** — Run queries and examine the results
3. **Test both execution paths** — Tool path (LLM → Tool → LLM) vs direct path (LLM → END)

## 1. Environment Setup & Graph Build

We rebuild the same graph from the previous notebook. The setup is compact since we've already learned each component.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from typing import Literal

print("✅ All imports successful")

In [ ]:
# Define tools
@tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """
    Convert currency from one type to another.
    
    Use this tool when users need to convert monetary amounts between
    different currencies. Supports USD, EUR, GBP, INR, and JPY.
    """
    exchange_rates = {"USD": 1.0, "EUR": 0.92, "GBP": 0.79, "INR": 83.12, "JPY": 149.50}
    
    from_currency = from_currency.upper()
    to_currency = to_currency.upper()
    
    if from_currency not in exchange_rates:
        return f"Error: Unsupported currency {from_currency}"
    if to_currency not in exchange_rates:
        return f"Error: Unsupported currency {to_currency}"
    
    amount_in_usd = amount / exchange_rates[from_currency]
    converted_amount = amount_in_usd * exchange_rates[to_currency]
    effective_rate = exchange_rates[to_currency] / exchange_rates[from_currency]
    
    return (
        f"Conversion Result:\n"
        f"  {amount:,.2f} {from_currency} = {converted_amount:,.2f} {to_currency}\n"
        f"  Exchange Rate: 1 {from_currency} = {effective_rate:.4f} {to_currency}"
    )

@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    """
    if principal <= 0:
        return "Error: Principal must be greater than 0"
    if annual_interest_rate < 0:
        return "Error: Interest rate cannot be negative"
    if tenure_months <= 0:
        return "Error: Tenure must be greater than 0"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ Tools defined")

In [ ]:
# Initialize LLM and bind tools
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

tools = [currency_converter, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

print("✅ LLM initialized with tools")

In [ ]:
# Build graph (same as previous notebook)
def call_llm(state: MessagesState):
    """LLM node that invokes the LLM."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router that decides next step based on tool_calls."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

print("✅ Graph built")

## 2. Understanding Compilation

Before we can use the graph, we must **compile** it. Compilation does two things:

1. **Validates the graph** — Checks that all nodes are reachable from START and all edges point to valid nodes
2. **Creates an executable** — Returns a `CompiledGraph` object with `invoke()` and `stream()` methods

Think of it like compiling code: you define the graph (write the source), then compile it (create the executable).

In [ ]:
app = workflow.compile()

print(f"Type: {type(app).__name__}")
print("✅ Graph compiled successfully")

### What happens if compilation fails?

Let's intentionally create an invalid graph to see the error.

In [ ]:
# Example: A graph with a node that has no incoming edge
bad_workflow = StateGraph(MessagesState)
bad_workflow.add_node("llm", call_llm)
bad_workflow.add_node("orphan", call_llm)  # No edge leads to this node
bad_workflow.add_edge(START, "llm")
bad_workflow.add_edge("llm", END)

try:
    bad_workflow.compile()
    print("Compiled (orphan nodes may be allowed but ignored)")
except Exception as e:
    print(f"Compilation error: {e}")

## 3. Invoking the Graph

The `invoke()` method takes an initial state and runs it through the graph until it reaches END.

```python
result = app.invoke({"messages": [HumanMessage(content="...")]})
```

The result is the **final state** containing all messages generated during execution.

## 4. Test: Tool Execution Path

When the query requires a tool, the graph follows: **START → llm → tools → llm → END**

In [ ]:
# Test 1: Currency conversion (requires tool)
result1 = app.invoke({
    "messages": [HumanMessage(content="Convert 500 USD to INR")]
})

print("TEST 1: Currency Conversion")
print("=" * 60)
print(f"Messages generated: {len(result1['messages'])}")
print(f"Path: START → llm → tools → llm → END")
print(f"\nFinal Response:")
print(result1['messages'][-1].content)

In [ ]:
# Test 2: EMI calculation (requires tool)
result2 = app.invoke({
    "messages": [HumanMessage(content="Calculate EMI for a 2,000,000 INR home loan at 8.5% for 20 years")]
})

print("TEST 2: EMI Calculation")
print("=" * 60)
print(f"Messages generated: {len(result2['messages'])}")
print(f"Path: START → llm → tools → llm → END")
print(f"\nFinal Response:")
print(result2['messages'][-1].content)

Both tool queries generate **4 messages**:

| # | Message Type | Created By |
|---|---|---|
| 1 | `HumanMessage` | User (our input) |
| 2 | `AIMessage` with `tool_calls` | LLM node (first pass) |
| 3 | `ToolMessage` | Tool node |
| 4 | `AIMessage` with `content` | LLM node (second pass) |

## 5. Test: Direct Path (No Tool)

When the query doesn't need a tool, the graph follows: **START → llm → END**

In [ ]:
# Test 3: General greeting (no tool needed)
result3 = app.invoke({
    "messages": [HumanMessage(content="Hello! What can you help me with?")]
})

print("TEST 3: General Greeting")
print("=" * 60)
print(f"Messages generated: {len(result3['messages'])}")
print(f"Path: START → llm → END")
print(f"\nFinal Response:")
print(result3['messages'][-1].content)

In [ ]:
# Test 4: General knowledge question (no tool needed)
result4 = app.invoke({
    "messages": [HumanMessage(content="What does EMI stand for?")]
})

print("TEST 4: General Knowledge")
print("=" * 60)
print(f"Messages generated: {len(result4['messages'])}")
print(f"Path: START → llm → END")
print(f"\nFinal Response:")
print(result4['messages'][-1].content)

Non-tool queries generate only **2 messages**:

| # | Message Type | Created By |
|---|---|---|
| 1 | `HumanMessage` | User (our input) |
| 2 | `AIMessage` with `content` | LLM node (responds directly) |

The router sees no `tool_calls` in the AIMessage, so it routes directly to END.

## 6. Comparing Both Paths

Let's summarize the message counts across all tests.

In [ ]:
# Summary comparison
tests = [
    ("Convert 500 USD to INR", result1),
    ("Calculate EMI for home loan", result2),
    ("Hello! What can you help me with?", result3),
    ("What does EMI stand for?", result4)
]

print("EXECUTION PATH COMPARISON")
print("=" * 70)
print(f"{'Query':<40} {'Messages':>10} {'Path':>18}")
print("-" * 70)

for query, result in tests:
    msg_count = len(result['messages'])
    # Check if any AIMessage has tool_calls
    has_tool = any(
        hasattr(m, 'tool_calls') and m.tool_calls 
        for m in result['messages'] if isinstance(m, AIMessage)
    )
    path = "llm → tools → llm" if has_tool else "llm → END"
    query_short = query[:38] + ".." if len(query) > 40 else query
    print(f"{query_short:<40} {msg_count:>10} {path:>18}")

print("=" * 70)

## 7. Understanding the Result State

The result from `invoke()` is a dictionary with the same structure as `MessagesState`. Let's inspect it.

In [ ]:
# Inspect the result structure
print(f"Result type: {type(result1)}")
print(f"Result keys: {list(result1.keys())}")
print(f"Messages type: {type(result1['messages'])}")
print(f"Messages count: {len(result1['messages'])}")

In [ ]:
# Inspect each message's type in the tool execution result
print("Message types in tool execution result:")
print("=" * 60)

for i, msg in enumerate(result1['messages']):
    msg_type = type(msg).__name__
    
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls:
        msg_type += " (with tool_calls)"
    elif isinstance(msg, AIMessage):
        msg_type += " (final response)"
    
    print(f"  [{i}] {msg_type}")

In [ ]:
# Inspect each message's type in the direct path result
print("Message types in direct path result:")
print("=" * 60)

for i, msg in enumerate(result3['messages']):
    msg_type = type(msg).__name__
    
    if isinstance(msg, AIMessage):
        msg_type += " (final response)"
    
    print(f"  [{i}] {msg_type}")

## Conclusion

### What You've Accomplished

✅ **Understood compilation** — `compile()` validates the graph and creates an executable application

✅ **Invoked the graph** — Used `invoke()` to run queries and examined the result state

✅ **Tested both execution paths:**
- **Tool path** (4 messages): HumanMessage → AIMessage with tool_calls → ToolMessage → AIMessage with content
- **Direct path** (2 messages): HumanMessage → AIMessage with content

### Key Insight

The **router function** is the decision point. It checks whether the LLM's response contains `tool_calls`. If yes, the graph loops through the tool node and back to the LLM. If no, the graph ends. This simple check enables the LLM to dynamically decide whether to use tools.

### Next Steps

Continue to **Notebook 075: Single Tool Execution** to deep-dive into the message flow and examine the internal structure of each message type.